## Understanding " NEURAL PROBABILISTIC LANGUAGE MODEL "

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:


# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]



['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [10]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words:
  
  #print(w)
  context = [0] * block_size
  
  for ch in w + '.':
    ix = stoi[ch]
   
    X.append(context)
   
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

In [57]:
X.shape, Y.shape
print(X)


tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]])


In [69]:
X[1][2]

tensor(5)

In [73]:
C = torch.randn((27,6))  # we are creating a less dimesion vector for inputing large amount of data
C[X]

tensor([[[ 0.3039,  0.8955,  1.0232, -0.8745,  1.0886, -0.2092],
         [ 0.3039,  0.8955,  1.0232, -0.8745,  1.0886, -0.2092],
         [ 0.3039,  0.8955,  1.0232, -0.8745,  1.0886, -0.2092]],

        [[ 0.3039,  0.8955,  1.0232, -0.8745,  1.0886, -0.2092],
         [ 0.3039,  0.8955,  1.0232, -0.8745,  1.0886, -0.2092],
         [-0.9541,  1.7158, -0.1388, -0.0129,  0.7623, -0.1952]],

        [[ 0.3039,  0.8955,  1.0232, -0.8745,  1.0886, -0.2092],
         [-0.9541,  1.7158, -0.1388, -0.0129,  0.7623, -0.1952],
         [-0.9274,  1.0568, -1.3406,  0.8325,  0.8657, -0.9434]],

        ...,

        [[-0.6245,  0.6163,  0.2477,  0.0116,  0.7227, -0.1402],
         [-0.6245,  0.6163,  0.2477,  0.0116,  0.7227, -0.1402],
         [ 0.0648,  1.1934,  0.9448, -0.0524, -0.9263,  0.1483]],

        [[-0.6245,  0.6163,  0.2477,  0.0116,  0.7227, -0.1402],
         [ 0.0648,  1.1934,  0.9448, -0.0524, -0.9263,  0.1483],
         [-0.6245,  0.6163,  0.2477,  0.0116,  0.7227, -0.1402]],

 

In [74]:
emb = C[X]
emb[:3].shape


torch.Size([3, 3, 6])

In [77]:
W1 = torch.randn((18, 100))
b1 = torch.randn(100)

In [78]:
h = torch.tanh(emb.view(-1, 18) @ W1 + b1)

In [79]:


h.shape




torch.Size([228146, 100])

In [80]:


W2 = torch.randn((100, 27))
b2 = torch.randn(27)



In [81]:


logits = h @ W2 + b2



In [82]:


logits.shape



torch.Size([228146, 27])

In [83]:
counts = logits.exp()


prob = counts / counts.sum(1, keepdims=True)



prob.shape







torch.Size([228146, 27])

In [84]:
loss = -prob[torch.arange(228146), Y].log().mean()
loss

tensor(18.5636)

In [ ]:
'''
This loss is computationally better than the loss that we have calculated above because of 
 --> less creation of tensors internally  
 --> It smoothens the tensors by subracting eith max tensor in that row to avoid the ouliers of large positive values

'''

loss = F.cross_entropy(logits, Y[ix])